# Configuration Notebook
Useful for debugging configurations and viewing project configuration details.

## Setup
Configure defaults and select a project.

In [ ]:
# Set defaults
#default_projects_directory = '/home/dinalt/ai_assets/projects/experiments'
default_projects_directory = '../examples/trainers'
default_project = "dynamic_models"
config_template = "pre_ln.yaml"

from ipyfilechooser import FileChooser
import os
fc = FileChooser(
    os.path.join(default_projects_directory, default_project), show_only_dirs=True,
    title="Select a Project Directory", select_default=True)
display(fc)

## Project Info

In [ ]:
import sys, os
modules_path = os.path.join('..', 'src')
if modules_path not in sys.path: sys.path.insert(0, modules_path)
from pprint import pformat, pp
from IPython import display as ds
from forgather.config import (
    ConfigEnvironment,
    pconfig
)
from aiws.config import preprocessor_globals, MetaConfig
import aiws.notebooks as nb

assert os.path.exists(fc.selected_path), "Project directory does not exist."
nb.show_project_readme(fc.selected_path)
meta = MetaConfig(fc.selected_path)
nb.display_meta(meta, "### Meta Config\n")
nb.list_templates(meta.find_templates(meta.config_prefix), "### Available Configurations\n")
default_config = meta.default_config()
print('-' * 60)
print(f"Default Configuration: {default_config}")

## List Available Templates
This will list all templates within the searchpath.

In [ ]:
def list_templates(prefix):
    nb.list_templates(meta.find_templates(prefix), "### Templates\n")
list_templates('')

## Configuration
Set 'config_template' to the name of a configuration template in the project. See: "Available Configurations"
If unspecified, the first configuration in the project will be used.

In [ ]:
# Create configuration envrionment
environment = ConfigEnvironment(
    searchpath=meta.searchpath,
    globals = preprocessor_globals(fc.selected_path),
)

config_template_path = meta.config_path(config_template)
nb.display_preprocessed_template(environment, config_template_path, title="### Preprocessed Configuration\n")
nb.display_referenced_templates_tree(environment, config_template_path, "### Included Templates\n")
config, pp_config = environment.load(config_template_path).get()
nb.display_referenced_source_list(config, "### Included Sources\n")
display(ds.Markdown("### Loaded Configuration\n"))
pconfig(config)

## List Sub-Modules
Show referenced sub-modules, within the same package, as the source files.
This requires loading the module dependencies, which may take a moment.
Don't run this, unless you trust the source.

For configurations with dynamic imports, the dynamically imported modules will not be resolvable until after the configuration has been instantiated.

In [ ]:
nb.display_referenced_source_list(config, title="Sub-Modules\n", deep=True)

## Materialized Configuration

Instantiate the configuration from the definition.
This loads all of the referenced modules and instantiates the main output. Some configurations will run preprocessing when loaded, so this can take a moment.

And don't run this if you don't trust the source of the configuration!

In [ ]:
config, pp_config = environment.load(config_template_path).get()

# Note: We inject the pre-processed config as an argument, which can then be used to log this information.
main_output = config.main(pp_config=pp_config)
pconfig(main_output)

### Run Configuration

Assuming that this the output object has a 'run' method (training scripts do), the following will run it.

For a more robust approach, see: [train.ipynb](train.ipynb)

In [ ]:
main_output.trainer.model

In [ ]:
main_output.run()

### Cleanup
Note: These will show the target directory and ask for confirmation before proceeding.

#### Delete All

In [ ]:
nb.delete_dir(config.meta['models_dir'] "Delete all models in project")

#### Delete Configuration Output Directory
This will delete the model and logs for the current configuration.

In [ ]:
nb.delete_dir(config.meta['output_dir'], "Delete output directory")